In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import tensorflow as tf
import keras.backend.tensorflow_backend as KTF

def get_session(gpu_fraction=0.3):
    '''Assume that you have 6GB of GPU memory and want to allocate ~2GB'''
    num_threads = os.environ.get('OMP_NUM_THREADS')
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=gpu_fraction)

    if num_threads:
        return tf.Session(config=tf.ConfigProto(
            gpu_options=gpu_options, intra_op_parallelism_threads=num_threads))
    else:
        return tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

KTF.set_session(get_session(0.5))

Using TensorFlow backend.


In [2]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Input
from keras.models import Model

from keras.layers import ZeroPadding2D,Convolution2D
import numpy as np

import os

In [3]:
batch_size = 16
num_classes = 3
epochs = 100

# num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_neuron_classification_trained_model.h5'

In [4]:
# write a function to select x_train y_train x_test y_test
data_x = np.load('image_data_256x256.npy')
data_y = np.load('labels.npy')
data_x = np.swapaxes(np.swapaxes(data_x,1,2),2,3)
data_y = np.array([data_y]).T

from sklearn.cross_validation import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data_x, data_y, test_size=0.2)

print x_train.shape, x_test.shape, y_train.shape, y_test.shape
# shuffle the data
# select 20% as the test data and 80% as the training data
# np.random.choice(len(data_x), len(data_x) * 0.2, replace=False)

/home/edward/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


(221, 256, 256, 2) (56, 256, 256, 2) (221, 1) (56, 1)


In [5]:
# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

print x_train.shape, x_test.shape, y_train.shape, y_test.shape

(221, 256, 256, 2) (56, 256, 256, 2) (221, 3) (56, 3)


In [6]:
img_input = Input(shape = (256, 256, 2))
# DIM_ORDERING = "tf"

# if DIM_ORDERING == 'tf':
#     x_train = np.transpose(x_train.astype('float32') / 255., (0, 1, 2, 3))
#     x_test = np.transpose(x_test.astype('float32') / 255., (0, 1, 2, 3))

# if DIM_ORDERING == 'th':
#     x_train = x_train.astype('float32') / 255.
#     x_test = x_test.astype('float32') / 255.
    
print x_train.shape, x_test.shape, y_train.shape, y_test.shape

(221, 256, 256, 2) (56, 256, 256, 2) (221, 3) (56, 3)


In [7]:
# define the model 
# CNN structure
# model = Sequential()
# model.add(Conv2D(8, (3, 3), padding='same',
#                  input_shape=x_train.shape[1:]))
# model.add(Activation('relu'))
# model.add(Conv2D(8, (3, 3)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))

# model.add(Conv2D(16, (3, 3), padding='same'))
# model.add(Activation('relu'))
# model.add(Conv2D(16, (3, 3)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))

# model.add(Flatten())
# model.add(Dense(256))
# model.add(Activation('relu'))
# model.add(Dropout(0.5))
# model.add(Dense(num_classes))
# model.add(Activation('softmax'))

model = Sequential()
model.add(Conv2D(64, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
          
model.add(Conv2D(128, (3, 3) , padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3) , padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(512, (3, 3) , padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(512, (3, 3)))
model.add(Activation('relu'))
model.add(Conv2D(512, (3, 3)))
model.add(Activation('relu'))
model.add(Conv2D(512, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(3))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(3))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(3))
model.add(Activation('softmax'))
          
          
# # Layer Cluster - 1
# x = ZeroPadding2D(padding=(1, 1), dim_ordering=DIM_ORDERING)(img_input)
# # x = Convolution2D(64, 3, 3,
# #                   activation='relu',
# #                   border_mode='same',
# #                   dim_ordering=DIM_ORDERING)(x)
# x = Convolution2D(16, 3, 3,
#                   activation='relu',
#                   border_mode='same',
#                   dim_ordering=DIM_ORDERING)(x)
# x = ZeroPadding2D(padding=(1, 1), dim_ordering=DIM_ORDERING)(x)
# # x = Convolution2D(64, 3, 3,
# #                   activation='relu',
# #                   border_mode='same',
# #                   dim_ordering=DIM_ORDERING)(x)
# x = Convolution2D(16, 3, 3,
#                   activation='relu',
#                   border_mode='same',
#                   dim_ordering=DIM_ORDERING)(x)
# x = MaxPooling2D((2, 2), strides=(2, 2))(x)

# # Layer Cluster - 2
# x = ZeroPadding2D(padding=(1, 1), dim_ordering=DIM_ORDERING)(x)
# x = Convolution2D(128, 3, 3,
#                   activation='relu',
#                   border_mode='same',
#                   dim_ordering=DIM_ORDERING)(x)
# x = ZeroPadding2D(padding=(1, 1), dim_ordering=DIM_ORDERING)(x)
# x = Convolution2D(128, 3, 3,
#                   activation='relu',
#                   border_mode='same',
#                   dim_ordering=DIM_ORDERING)(x)
# x = MaxPooling2D((2, 2), strides=(2, 2))(x)

# # Layer Cluster - 3
# x = ZeroPadding2D(padding=(1, 1), dim_ordering=DIM_ORDERING)(x)
# x = Convolution2D(256, 3, 3,
#                   activation='relu',
#                   border_mode='same',
#                   dim_ordering=DIM_ORDERING)(x)
# x = ZeroPadding2D(padding=(1, 1), dim_ordering=DIM_ORDERING)(x)
# x = Convolution2D(256, 3, 3,
#                   activation='relu',
#                   border_mode='same',
#                   dim_ordering=DIM_ORDERING)(x)
# x = ZeroPadding2D(padding=(1, 1), dim_ordering=DIM_ORDERING)(x)
# x = Convolution2D(256, 3, 3,
#                   activation='relu',
#                   border_mode='same',
#                   dim_ordering=DIM_ORDERING)(x)
# x = ZeroPadding2D(padding=(1, 1), dim_ordering=DIM_ORDERING)(x)
# x = Convolution2D(256, 3, 3,
#                   activation='relu',
#                   border_mode='same',
#                   dim_ordering=DIM_ORDERING)(x)
# x = MaxPooling2D((2, 2), strides=(2, 2))(x)

# # Layer Cluster - 4
# x = ZeroPadding2D(padding=(1, 1), dim_ordering=DIM_ORDERING)(x)
# x = Convolution2D(512, 3, 3,
#                   activation='relu',
#                   border_mode='same',
#                   dim_ordering=DIM_ORDERING)(x)
# x = ZeroPadding2D(padding=(1, 1), dim_ordering=DIM_ORDERING)(x)
# x = Convolution2D(512, 3, 3,
#                   activation='relu',
#                   border_mode='same',
#                   dim_ordering=DIM_ORDERING)(x)
# x = ZeroPadding2D(padding=(1, 1), dim_ordering=DIM_ORDERING)(x)
# x = Convolution2D(512, 3, 3,
#                   activation='relu',
#                   border_mode='same',
#                   dim_ordering=DIM_ORDERING)(x)
# x = ZeroPadding2D(padding=(1, 1), dim_ordering=DIM_ORDERING)(x)
# x = Convolution2D(512, 3, 3,
#                   activation='relu',
#                   border_mode='same',
#                   dim_ordering=DIM_ORDERING)(x)
# x = MaxPooling2D((2, 2), strides=(2, 2))(x)

# # Layer Cluster - 5 - Output Layer
# x = Flatten()(x)
# x = Dense(num_classes, activation='relu')(x)
# x = Dropout(0.5)(x)
# x = Dense(num_classes, activation='relu')(x)
# x = Dropout(0.5)(x)
# x = Dense(num_classes, activation='softmax')(x)

In [8]:
# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])
model.summary()

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

print x_train.shape, x_test.shape, y_train.shape, y_test.shape
print type(x_train),type(x_test),type(y_train),type(y_test)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 256, 256, 64)      1216      
_________________________________________________________________
activation_1 (Activation)    (None, 256, 256, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 254, 254, 64)      36928     
_________________________________________________________________
activation_2 (Activation)    (None, 254, 254, 64)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 127, 127, 64)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 127, 127, 128)     73856     
_________________________________________________________________
activation_3 (Activation)    (None, 127, 127, 128)     0         
__________

In [9]:
data_augmentation = True
epochs = 1000
batch_size = 8

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    for _ in range(1):
        model.fit_generator(datagen.flow(x_train, y_train,
                                         batch_size=batch_size),
                            steps_per_epoch=x_train.shape[0] // batch_size,
                            epochs=epochs,
                            validation_data=(x_test, y_test),
                            workers=4,
                            callbacks=None,
                            verbose=1)
#         print model.evaluate(x=x_test, y=y_test, batch_size=batch_size, verbose=1)
#         print( "\n[INFO] accuracy: {:.2f}%".format(accuracy * 100) )

Using real-time data augmentation.


/home/edward/anaconda2/lib/python2.7/site-packages/keras/preprocessing/image.py:653: UserWarning: Expected input to be images (as Numpy array) following the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3 or 4 channels on axis 3. However, it was passed an array with shape (221, 256, 256, 2) (2 channels).
  ' (' + str(x.shape[self.channel_axis]) + ' channels).')
/home/edward/anaconda2/lib/python2.7/site-packages/keras/preprocessing/image.py:787: UserWarning: NumpyArrayIterator is set to use the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3 or 4 channels on axis 3. However, it was passed an array with shape (221, 256, 256, 2) (2 channels).
  ' (' + str(self.x.shape[channels_axis]) + ' channels).')


Epoch 1/1000
27/27 [==============================] - 21s - loss: 1.1652 - acc: 0.4890 - val_loss: 1.0908 - val_acc: 0.5714
Epoch 2/1000
27/27 [==============================] - 15s - loss: 1.0962 - acc: 0.5139 - val_loss: 1.0879 - val_acc: 0.5714
Epoch 3/1000
27/27 [==============================] - 15s - loss: 1.0979 - acc: 0.5063 - val_loss: 1.0891 - val_acc: 0.5714
Epoch 4/1000
27/27 [==============================] - 15s - loss: 1.1028 - acc: 0.5083 - val_loss: 1.0869 - val_acc: 0.5714
Epoch 5/1000
27/27 [==============================] - 15s - loss: 1.0870 - acc: 0.5241 - val_loss: 1.0841 - val_acc: 0.5714
Epoch 6/1000
27/27 [==============================] - 15s - loss: 1.0880 - acc: 0.5500 - val_loss: 1.0799 - val_acc: 0.5714
Epoch 7/1000
27/27 [==============================] - 15s - loss: 1.0860 - acc: 0.4971 - val_loss: 1.0835 - val_acc: 0.5714
Epoch 8/1000
27/27 [==============================] - 15s - loss: 1.0793 - acc: 0.5817 - val_loss: 1.0740 - val_acc: 0.5714
Epoch 9/

27/27 [==============================] - 15s - loss: 1.0222 - acc: 0.6029 - val_loss: 1.0187 - val_acc: 0.5714
Epoch 67/1000
27/27 [==============================] - 14s - loss: 1.0038 - acc: 0.5824 - val_loss: 1.0120 - val_acc: 0.5714
Epoch 68/1000
27/27 [==============================] - 14s - loss: 1.0200 - acc: 0.6029 - val_loss: 1.0183 - val_acc: 0.5714
Epoch 69/1000
27/27 [==============================] - 14s - loss: 1.0086 - acc: 0.6063 - val_loss: 1.0190 - val_acc: 0.5714
Epoch 70/1000
27/27 [==============================] - 14s - loss: 1.0054 - acc: 0.5982 - val_loss: 1.0106 - val_acc: 0.5714
Epoch 71/1000
27/27 [==============================] - 14s - loss: 1.0285 - acc: 0.5692 - val_loss: 1.0146 - val_acc: 0.5714
Epoch 72/1000
27/27 [==============================] - 14s - loss: 0.9942 - acc: 0.6334 - val_loss: 1.0119 - val_acc: 0.5714
Epoch 73/1000
27/27 [==============================] - 14s - loss: 1.0135 - acc: 0.6036 - val_loss: 1.0143 - val_acc: 0.5714
Epoch 74/1000


27/27 [==============================] - 14s - loss: 0.9512 - acc: 0.6433 - val_loss: 0.9919 - val_acc: 0.5714
Epoch 132/1000
27/27 [==============================] - 14s - loss: 0.9914 - acc: 0.5592 - val_loss: 0.9917 - val_acc: 0.5714
Epoch 133/1000
27/27 [==============================] - 14s - loss: 0.9731 - acc: 0.6002 - val_loss: 0.9899 - val_acc: 0.5714
Epoch 134/1000
27/27 [==============================] - 14s - loss: 0.9721 - acc: 0.6175 - val_loss: 0.9907 - val_acc: 0.5714
Epoch 135/1000
27/27 [==============================] - 14s - loss: 0.9979 - acc: 0.5870 - val_loss: 0.9916 - val_acc: 0.5714
Epoch 136/1000
27/27 [==============================] - 14s - loss: 0.9890 - acc: 0.5936 - val_loss: 0.9917 - val_acc: 0.5714
Epoch 137/1000
27/27 [==============================] - 14s - loss: 0.9678 - acc: 0.6195 - val_loss: 0.9917 - val_acc: 0.5714
Epoch 138/1000
27/27 [==============================] - 14s - loss: 0.9836 - acc: 0.5751 - val_loss: 0.9895 - val_acc: 0.5714
Epoch 1

...

27/27 [==============================] - 14s - loss: 0.8833 - acc: 0.6638 - val_loss: 0.9874 - val_acc: 0.5714
Epoch 717/1000
27/27 [==============================] - 14s - loss: 0.9858 - acc: 0.5705 - val_loss: 0.9873 - val_acc: 0.5714
Epoch 718/1000
27/27 [==============================] - 14s - loss: 0.9620 - acc: 0.5870 - val_loss: 0.9873 - val_acc: 0.5714
Epoch 719/1000
27/27 [==============================] - 14s - loss: 0.9095 - acc: 0.6360 - val_loss: 0.9874 - val_acc: 0.5714
Epoch 720/1000
27/27 [==============================] - 14s - loss: 0.9755 - acc: 0.5758 - val_loss: 0.9873 - val_acc: 0.5714
Epoch 721/1000
27/27 [==============================] - 14s - loss: 0.9408 - acc: 0.6056 - val_loss: 0.9873 - val_acc: 0.5714
Epoch 722/1000
27/27 [==============================] - 14s - loss: 0.9965 - acc: 0.5592 - val_loss: 0.9873 - val_acc: 0.5714
Epoch 723/1000
27/27 [==============================] - 14s - loss: 0.9329 - acc: 0.6129 - val_loss: 0.9874 - val_acc: 0.5714
Epoch 7

27/27 [==============================] - 14s - loss: 0.9605 - acc: 0.5936 - val_loss: 0.9877 - val_acc: 0.5714
Epoch 782/1000
27/27 [==============================] - 14s - loss: 0.9443 - acc: 0.6056 - val_loss: 0.9877 - val_acc: 0.5714
Epoch 783/1000
27/27 [==============================] - 14s - loss: 0.9759 - acc: 0.5731 - val_loss: 0.9877 - val_acc: 0.5714
Epoch 784/1000
27/27 [==============================] - 14s - loss: 0.9431 - acc: 0.6102 - val_loss: 0.9877 - val_acc: 0.5714
Epoch 785/1000
27/27 [==============================] - 14s - loss: 0.9575 - acc: 0.5970 - val_loss: 0.9877 - val_acc: 0.5714
Epoch 786/1000
27/27 [==============================] - 15s - loss: 0.9377 - acc: 0.6065 - val_loss: 0.9877 - val_acc: 0.5714
Epoch 787/1000
27/27 [==============================] - 14s - loss: 0.9668 - acc: 0.5870 - val_loss: 0.9876 - val_acc: 0.5714
Epoch 788/1000
27/27 [==============================] - 14s - loss: 0.9506 - acc: 0.6009 - val_loss: 0.9877 - val_acc: 0.5714
Epoch 7

27/27 [==============================] - 14s - loss: 0.9313 - acc: 0.6102 - val_loss: 0.9878 - val_acc: 0.5714
Epoch 847/1000
27/27 [==============================] - 14s - loss: 0.9476 - acc: 0.5982 - val_loss: 0.9878 - val_acc: 0.5714
Epoch 848/1000
27/27 [==============================] - 14s - loss: 0.9597 - acc: 0.5890 - val_loss: 0.9879 - val_acc: 0.5714
Epoch 849/1000
27/27 [==============================] - 14s - loss: 0.9544 - acc: 0.5936 - val_loss: 0.9879 - val_acc: 0.5714
Epoch 850/1000
27/27 [==============================] - 14s - loss: 0.9465 - acc: 0.6056 - val_loss: 0.9879 - val_acc: 0.5714
Epoch 851/1000
27/27 [==============================] - 14s - loss: 0.9604 - acc: 0.5870 - val_loss: 0.9878 - val_acc: 0.5714
Epoch 852/1000
27/27 [==============================] - 14s - loss: 0.9233 - acc: 0.6287 - val_loss: 0.9878 - val_acc: 0.5714
Epoch 853/1000
27/27 [==============================] - 14s - loss: 0.9684 - acc: 0.5770 - val_loss: 0.9879 - val_acc: 0.5714
Epoch 8

27/27 [==============================] - 14s - loss: 0.9795 - acc: 0.5817 - val_loss: 0.9881 - val_acc: 0.5714
Epoch 912/1000
27/27 [==============================] - 14s - loss: 0.9179 - acc: 0.6287 - val_loss: 0.9881 - val_acc: 0.5714
Epoch 913/1000
27/27 [==============================] - 14s - loss: 0.9543 - acc: 0.5963 - val_loss: 0.9881 - val_acc: 0.5714
Epoch 914/1000
27/27 [==============================] - 14s - loss: 0.9297 - acc: 0.6202 - val_loss: 0.9881 - val_acc: 0.5714
Epoch 915/1000
27/27 [==============================] - 14s - loss: 0.9875 - acc: 0.5566 - val_loss: 0.9881 - val_acc: 0.5714
Epoch 916/1000
27/27 [==============================] - 14s - loss: 0.9255 - acc: 0.6241 - val_loss: 0.9881 - val_acc: 0.5714
Epoch 917/1000
27/27 [==============================] - 14s - loss: 0.9733 - acc: 0.5751 - val_loss: 0.9881 - val_acc: 0.5714
Epoch 918/1000
27/27 [==============================] - 14s - loss: 0.9571 - acc: 0.5878 - val_loss: 0.9881 - val_acc: 0.5714
Epoch 9

27/27 [==============================] - 14s - loss: 0.9613 - acc: 0.5936 - val_loss: 0.9882 - val_acc: 0.5714
Epoch 977/1000
27/27 [==============================] - 14s - loss: 0.9557 - acc: 0.5936 - val_loss: 0.9881 - val_acc: 0.5714
Epoch 978/1000
27/27 [==============================] - 14s - loss: 0.9454 - acc: 0.5982 - val_loss: 0.9882 - val_acc: 0.5714
Epoch 979/1000
27/27 [==============================] - 14s - loss: 0.9585 - acc: 0.5963 - val_loss: 0.9882 - val_acc: 0.5714
Epoch 980/1000
27/27 [==============================] - 14s - loss: 0.9593 - acc: 0.5917 - val_loss: 0.9882 - val_acc: 0.5714
Epoch 981/1000
27/27 [==============================] - 14s - loss: 0.9587 - acc: 0.5863 - val_loss: 0.9882 - val_acc: 0.5714
Epoch 982/1000
27/27 [==============================] - 15s - loss: 0.9420 - acc: 0.6111 - val_loss: 0.9882 - val_acc: 0.5714
Epoch 983/1000
27/27 [==============================] - 14s - loss: 0.9416 - acc: 0.6056 - val_loss: 0.9882 - val_acc: 0.5714
Epoch 9